In [1]:
import numpy as np
import os
import random

from os.path import join as pjoin
from collections import defaultdict

from keras.layers import Dense, Input, BatchNormalization, Activation
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler

from sklearn.ensemble import RandomForestClassifier

Using TensorFlow backend.


In [2]:
base = 24
datadir = 'data'
dataset_size=1e6
filename = 'Base{}.txt'.format(base)
train_split=0.7
val_split=0.2
test_split=0.1
batch_size=500
lr = 1e-2

In [3]:
def get_data():
    data = []
    for row in open(pjoin(datadir, filename)):
        if len(row.split()) == 2:
            inp, target = row.split()
            data.append([np.array([float(x) for x in inp]), float(target)])
    random.shuffle(data)
    train_count = int(dataset_size*train_split)
    val_count = int(dataset_size*val_split)
    test_count = int(dataset_size*test_split)
    return data[:train_count], data[train_count:train_count+val_count], \
           data[train_count+val_count:train_count+val_count+test_count]

In [4]:
def generator(data):
    while 1:
        X, Y = np.zeros([batch_size, base]), np.zeros([batch_size, 1])
        random.shuffle(data)
        for i, (x, y) in enumerate(data):
            X[i%batch_size] = x
            Y[i%batch_size] = y
            if (i + 1) % batch_size == 0:
                yield X, Y
                X, Y = np.zeros([batch_size, base]), np.zeros([batch_size, 1])

In [5]:
train_data, val_data, test_data = get_data()
train_generator, val_generator, test_generator = generator(train_data), generator(val_data), generator(test_data)

In [6]:
def nn_model():
    optimizer = RMSprop(lr)
    inp = Input((base,))
    x = Dense(1, activation='sigmoid')(inp)
    model = Model(inp, x)
    model.compile(loss='binary_crossentropy', metrics=['binary_accuracy'], optimizer=optimizer)
    return model
    

In [7]:
def step_decay_schedule(initial_lr=1e-2, decay_factor=0.9999, min_lr=1e-6):
    def schedule(epoch):
        return max(min_lr, initial_lr * (decay_factor ** (epoch)))
    
    return LearningRateScheduler(schedule)

In [8]:
model = nn_model()
#plateau_callback = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=10, verbose=1, min_lr=1e-6)
model.fit_generator(train_generator, 
                    steps_per_epoch=int(train_split*dataset_size / batch_size),
                    epochs=100000,
                    validation_data=val_generator,
                    validation_steps=int(val_split*dataset_size / batch_size),
                    callbacks = [step_decay_schedule()])

Epoch 1/100000
1400/1400 [==============================] - 7s 5ms/step - loss: 0.6637 - binary_accuracy: 0.6234 - val_loss: 0.6624 - val_binary_accuracy: 0.6261
Epoch 2/100000
1400/1400 [==============================] - 6s 4ms/step - loss: 0.6625 - binary_accuracy: 0.6254 - val_loss: 0.6621 - val_binary_accuracy: 0.6261
Epoch 3/100000
1400/1400 [==============================] - 6s 4ms/step - loss: 0.6625 - binary_accuracy: 0.6254 - val_loss: 0.6625 - val_binary_accuracy: 0.6261
Epoch 4/100000
1400/1400 [==============================] - 6s 4ms/step - loss: 0.6625 - binary_accuracy: 0.6254 - val_loss: 0.6613 - val_binary_accuracy: 0.6261
Epoch 5/100000
1400/1400 [==============================] - 6s 4ms/step - loss: 0.6624 - binary_accuracy: 0.6254 - val_loss: 0.6620 - val_binary_accuracy: 0.6261
Epoch 6/100000
1400/1400 [==============================] - 6s 4ms/step - loss: 0.6625 - binary_accuracy: 0.6254 - val_loss: 0.6621 - val_binary_accuracy: 0.6261
Epoch 7/100000
1400/1400 [==

KeyboardInterrupt: 

In [9]:
x, y = zip(*train_data)
x_val, y_val = zip(*test_data)
x_test, y_test = zip(*test_data)

In [10]:
clf = RandomForestClassifier(n_estimators = 1000, random_state = 42, n_jobs = -1, verbose=1)
# Train the model on training data
print("data loaded")
clf.fit(x, y);

data loaded


KeyboardInterrupt: 

In [10]:
res = clf.predict(x_test)
print(1 - np.abs(np.array(res) - np.array(y_test)).sum() / len(y_test))


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    2.5s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   12.5s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   33.3s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:  1.4min finished


0.67073


In [10]:
def reset_lr():
    def callback(env):
        env.params['learning_rate'] = max(env.params['learning_rate']*0.9999, 1e-6)
    callback.before_iteration = True
    callback.order = 0
    return callback

In [ ]:
import lightgbm as lgb

train_data = lgb.Dataset(np.array(x), label=np.array(y), \
                         feature_name=['c{}'.format(i) for i in range(base)], \
                         categorical_feature=['c{}'.format(i) for i in range(base)])
val_data = lgb.Dataset(np.array(x_val), label=np.array(y_val), \
                         feature_name=['c{}'.format(i) for i in range(base)], \
                         categorical_feature=['c{}'.format(i) for i in range(base)])
param = {'num_leaves': 1023, 
         'num_trees':100000, 
         'objective':'binary', 
         'learning_rate' : lr,
         'boosting': 'dart',
         'max_bin': 1023}
param['metric'] = ['auc', 'binary_logloss']

num_round = 1000
bst = lgb.train(param, train_data, num_round, valid_sets=[val_data], \
                callbacks=[reset_lr()])

/usr/local/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
/usr/local/lib/python3.6/site-packages/lightgbm/engine.py:113: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.6/site-packages/lightgbm/basic.py:1184: UserWarning: Using cate

[1]	valid_0's binary_logloss: 0.692207	valid_0's auc: 0.670667
[2]	valid_0's binary_logloss: 0.691284	valid_0's auc: 0.670937
[3]	valid_0's binary_logloss: 0.690378	valid_0's auc: 0.671085
[4]	valid_0's binary_logloss: 0.689362	valid_0's auc: 0.677791
[5]	valid_0's binary_logloss: 0.688366	valid_0's auc: 0.680906
[6]	valid_0's binary_logloss: 0.68737	valid_0's auc: 0.682907
[7]	valid_0's binary_logloss: 0.686555	valid_0's auc: 0.682383
[8]	valid_0's binary_logloss: 0.686846	valid_0's auc: 0.682728
[9]	valid_0's binary_logloss: 0.685891	valid_0's auc: 0.683823
[10]	valid_0's binary_logloss: 0.68497	valid_0's auc: 0.684611
[11]	valid_0's binary_logloss: 0.684042	valid_0's auc: 0.685325
[12]	valid_0's binary_logloss: 0.684359	valid_0's auc: 0.685179
[13]	valid_0's binary_logloss: 0.683448	valid_0's auc: 0.685825
[14]	valid_0's binary_logloss: 0.682555	valid_0's auc: 0.686135
[15]	valid_0's binary_logloss: 0.68168	valid_0's auc: 0.686468
[16]	valid_0's binary_logloss: 0.680824	valid_0's au

[131]	valid_0's binary_logloss: 0.657864	valid_0's auc: 0.699869
[132]	valid_0's binary_logloss: 0.657027	valid_0's auc: 0.702397
[133]	valid_0's binary_logloss: 0.656102	valid_0's auc: 0.704758
[134]	valid_0's binary_logloss: 0.65636	valid_0's auc: 0.704819
[135]	valid_0's binary_logloss: 0.655904	valid_0's auc: 0.704889
[136]	valid_0's binary_logloss: 0.656215	valid_0's auc: 0.70478
[137]	valid_0's binary_logloss: 0.656457	valid_0's auc: 0.704723
[138]	valid_0's binary_logloss: 0.655987	valid_0's auc: 0.70457
[139]	valid_0's binary_logloss: 0.655001	valid_0's auc: 0.707518
[140]	valid_0's binary_logloss: 0.655284	valid_0's auc: 0.707566
[141]	valid_0's binary_logloss: 0.65552	valid_0's auc: 0.707613
[142]	valid_0's binary_logloss: 0.655792	valid_0's auc: 0.707521
[143]	valid_0's binary_logloss: 0.655305	valid_0's auc: 0.707396
[144]	valid_0's binary_logloss: 0.655515	valid_0's auc: 0.70745
[145]	valid_0's binary_logloss: 0.654538	valid_0's auc: 0.710064
[146]	valid_0's binary_logloss

[258]	valid_0's binary_logloss: 0.628246	valid_0's auc: 0.745066
[259]	valid_0's binary_logloss: 0.628549	valid_0's auc: 0.745026
[260]	valid_0's binary_logloss: 0.628869	valid_0's auc: 0.744983
[261]	valid_0's binary_logloss: 0.629154	valid_0's auc: 0.744959
[262]	valid_0's binary_logloss: 0.628402	valid_0's auc: 0.74576
[263]	valid_0's binary_logloss: 0.627955	valid_0's auc: 0.746201
[264]	valid_0's binary_logloss: 0.627348	valid_0's auc: 0.746558
[265]	valid_0's binary_logloss: 0.626291	valid_0's auc: 0.748184
[266]	valid_0's binary_logloss: 0.625407	valid_0's auc: 0.749269
[267]	valid_0's binary_logloss: 0.625717	valid_0's auc: 0.749189
[268]	valid_0's binary_logloss: 0.624853	valid_0's auc: 0.750197
[269]	valid_0's binary_logloss: 0.625155	valid_0's auc: 0.750164
[270]	valid_0's binary_logloss: 0.624174	valid_0's auc: 0.751445
[271]	valid_0's binary_logloss: 0.624522	valid_0's auc: 0.751318
[272]	valid_0's binary_logloss: 0.623675	valid_0's auc: 0.752228
[273]	valid_0's binary_log

[385]	valid_0's binary_logloss: 0.606815	valid_0's auc: 0.775831
[386]	valid_0's binary_logloss: 0.606649	valid_0's auc: 0.775658
[387]	valid_0's binary_logloss: 0.60556	valid_0's auc: 0.777404
[388]	valid_0's binary_logloss: 0.60439	valid_0's auc: 0.779083
[389]	valid_0's binary_logloss: 0.603591	valid_0's auc: 0.779811
[390]	valid_0's binary_logloss: 0.602454	valid_0's auc: 0.781376
[391]	valid_0's binary_logloss: 0.60276	valid_0's auc: 0.781296
[392]	valid_0's binary_logloss: 0.60198	valid_0's auc: 0.781971
[393]	valid_0's binary_logloss: 0.600865	valid_0's auc: 0.783463
[394]	valid_0's binary_logloss: 0.599774	valid_0's auc: 0.784851
[395]	valid_0's binary_logloss: 0.599029	valid_0's auc: 0.785443
[396]	valid_0's binary_logloss: 0.599328	valid_0's auc: 0.785377
[397]	valid_0's binary_logloss: 0.598209	valid_0's auc: 0.786866
[398]	valid_0's binary_logloss: 0.597158	valid_0's auc: 0.788124
[399]	valid_0's binary_logloss: 0.59645	valid_0's auc: 0.788634
[400]	valid_0's binary_logloss

...

[3153]	valid_0's binary_logloss: 0.463595	valid_0's auc: 0.874451
[3154]	valid_0's binary_logloss: 0.463523	valid_0's auc: 0.874461
[3155]	valid_0's binary_logloss: 0.463489	valid_0's auc: 0.874452
[3156]	valid_0's binary_logloss: 0.463467	valid_0's auc: 0.874459
[3157]	valid_0's binary_logloss: 0.463503	valid_0's auc: 0.87446
[3158]	valid_0's binary_logloss: 0.463424	valid_0's auc: 0.874466
[3159]	valid_0's binary_logloss: 0.463459	valid_0's auc: 0.874469
[3160]	valid_0's binary_logloss: 0.463493	valid_0's auc: 0.874471
[3161]	valid_0's binary_logloss: 0.463543	valid_0's auc: 0.874471
[3162]	valid_0's binary_logloss: 0.463428	valid_0's auc: 0.874493
[3163]	valid_0's binary_logloss: 0.463466	valid_0's auc: 0.874494
[3164]	valid_0's binary_logloss: 0.463452	valid_0's auc: 0.874491
[3165]	valid_0's binary_logloss: 0.463333	valid_0's auc: 0.874521
[3166]	valid_0's binary_logloss: 0.463384	valid_0's auc: 0.874521
[3167]	valid_0's binary_logloss: 0.463417	valid_0's auc: 0.874522
[3168]	vali

[3278]	valid_0's binary_logloss: 0.462742	valid_0's auc: 0.874833
[3279]	valid_0's binary_logloss: 0.462775	valid_0's auc: 0.874834
[3280]	valid_0's binary_logloss: 0.462757	valid_0's auc: 0.874837
[3281]	valid_0's binary_logloss: 0.462745	valid_0's auc: 0.874839
[3282]	valid_0's binary_logloss: 0.462731	valid_0's auc: 0.874843
[3283]	valid_0's binary_logloss: 0.462762	valid_0's auc: 0.874844
[3284]	valid_0's binary_logloss: 0.462805	valid_0's auc: 0.874845
[3285]	valid_0's binary_logloss: 0.462791	valid_0's auc: 0.874848
[3286]	valid_0's binary_logloss: 0.46283	valid_0's auc: 0.874848
[3287]	valid_0's binary_logloss: 0.462876	valid_0's auc: 0.874849
[3288]	valid_0's binary_logloss: 0.462909	valid_0's auc: 0.87485
[3289]	valid_0's binary_logloss: 0.462815	valid_0's auc: 0.874863
[3290]	valid_0's binary_logloss: 0.462714	valid_0's auc: 0.87487
[3291]	valid_0's binary_logloss: 0.462766	valid_0's auc: 0.874872
[3292]	valid_0's binary_logloss: 0.46275	valid_0's auc: 0.874871
[3293]	valid_0

[3403]	valid_0's binary_logloss: 0.461325	valid_0's auc: 0.875219
[3404]	valid_0's binary_logloss: 0.461223	valid_0's auc: 0.87523
[3405]	valid_0's binary_logloss: 0.461219	valid_0's auc: 0.875223
[3406]	valid_0's binary_logloss: 0.461204	valid_0's auc: 0.87523
[3407]	valid_0's binary_logloss: 0.46113	valid_0's auc: 0.875241
[3408]	valid_0's binary_logloss: 0.46116	valid_0's auc: 0.875242
[3409]	valid_0's binary_logloss: 0.461187	valid_0's auc: 0.875244
[3410]	valid_0's binary_logloss: 0.461173	valid_0's auc: 0.875249
[3411]	valid_0's binary_logloss: 0.46116	valid_0's auc: 0.875253
[3412]	valid_0's binary_logloss: 0.461202	valid_0's auc: 0.875255
[3413]	valid_0's binary_logloss: 0.461117	valid_0's auc: 0.875267
[3414]	valid_0's binary_logloss: 0.461107	valid_0's auc: 0.875267
[3415]	valid_0's binary_logloss: 0.461043	valid_0's auc: 0.875277
[3416]	valid_0's binary_logloss: 0.461075	valid_0's auc: 0.875278
[3417]	valid_0's binary_logloss: 0.461067	valid_0's auc: 0.875279
[3418]	valid_0'

[3528]	valid_0's binary_logloss: 0.460753	valid_0's auc: 0.875569
[3529]	valid_0's binary_logloss: 0.460746	valid_0's auc: 0.875566
[3530]	valid_0's binary_logloss: 0.460659	valid_0's auc: 0.875577
[3531]	valid_0's binary_logloss: 0.460563	valid_0's auc: 0.875598
[3532]	valid_0's binary_logloss: 0.460554	valid_0's auc: 0.875595
[3533]	valid_0's binary_logloss: 0.460579	valid_0's auc: 0.875597
[3534]	valid_0's binary_logloss: 0.46061	valid_0's auc: 0.875598
[3535]	valid_0's binary_logloss: 0.460591	valid_0's auc: 0.875603
[3536]	valid_0's binary_logloss: 0.46063	valid_0's auc: 0.875605
[3537]	valid_0's binary_logloss: 0.460575	valid_0's auc: 0.875609
[3538]	valid_0's binary_logloss: 0.460492	valid_0's auc: 0.87562
[3539]	valid_0's binary_logloss: 0.460488	valid_0's auc: 0.875613
[3540]	valid_0's binary_logloss: 0.460475	valid_0's auc: 0.875612
[3541]	valid_0's binary_logloss: 0.460406	valid_0's auc: 0.875615
[3542]	valid_0's binary_logloss: 0.460397	valid_0's auc: 0.875612
[3543]	valid_

[3655]	valid_0's binary_logloss: 0.459434	valid_0's auc: 0.875994
[3656]	valid_0's binary_logloss: 0.45947	valid_0's auc: 0.875995
[3657]	valid_0's binary_logloss: 0.45951	valid_0's auc: 0.875996
[3658]	valid_0's binary_logloss: 0.459536	valid_0's auc: 0.875997
[3659]	valid_0's binary_logloss: 0.459568	valid_0's auc: 0.875998
[3660]	valid_0's binary_logloss: 0.459557	valid_0's auc: 0.875997
[3661]	valid_0's binary_logloss: 0.459449	valid_0's auc: 0.87601
[3662]	valid_0's binary_logloss: 0.459439	valid_0's auc: 0.876009
[3663]	valid_0's binary_logloss: 0.459467	valid_0's auc: 0.87601
[3664]	valid_0's binary_logloss: 0.459408	valid_0's auc: 0.876003
[3665]	valid_0's binary_logloss: 0.459446	valid_0's auc: 0.876004
[3666]	valid_0's binary_logloss: 0.459478	valid_0's auc: 0.876005
[3667]	valid_0's binary_logloss: 0.459464	valid_0's auc: 0.876006
[3668]	valid_0's binary_logloss: 0.459362	valid_0's auc: 0.876016
[3669]	valid_0's binary_logloss: 0.459386	valid_0's auc: 0.876017
[3670]	valid_0